# BRAPI Database DEMO

# Testing DMS BrAPI connections

In [ ]:
library(magrittr)
library(rTASSEL)
library(tidyverse)

## Source code

In [ ]:
# handled by Extension
# source("https://raw.githubusercontent.com/agostof/BrAPI-Provider/master/BrapiProvider.R")

# You can use the following functions to interact with BMS
# see the QBMS documentation for more details.
# https://icarda-git.github.io/QBMS/

# pheno_provider$set_crop()
# pheno_provider$list_programs()
# pheno_provider$set_program()
# pheno_provider$list_trials()
# pheno_provider$set_trial()
# pheno_provider$list_studies()
# pheno_provider$set_study(study_name)
# pheno_provider$get_study_data()

#
# You can check the TOKENs with:
# pheno_provider$get_token()
# pheno_provider$list_crops()


# You can use the following functions to interact with GIGWA
# see the QBMS documentation for more details.
# geno_provider$gigwa_list_dbs()
# geno_provider$gigwa_set_db()
# geno_provider$gigwa_set_project()
# geno_samples = geno_provider$gigwa_get_samples()

#
# You can check the TOKENs with:
# geno_provider$get_token()
# geno_provider$gigwa_list_dbs()

## Build phenotype and genotype connection objects

In [ ]:
# # handled by Extension
# pheno_provider <- BrapiBms$new(url = "https://bms.ilci.scienceversa.com")
# pheno_provider$login(username=pheno_user, password=pheno_pass)

In [ ]:
# geno_provider <- BrapiGigwa$new(url="https://gigwa.ilci.scienceversa.com")
# geno_provider$login(username=geno_user, password=geno_pass)

## Retrieve genotype data (Gigwa)

In [ ]:
geno_provider$gigwa_list_dbs()

In [ ]:
geno_provider$gigwa_set_db("rtassel_demo")

In [ ]:
geno_provider$gigwa_set_project("rtassel_hmp")

In [ ]:
geno_samples = geno_provider$gigwa_get_samples()

In [ ]:
# will restrict samples for demo purposes
# genoDataFromGigwa <- geno_provider$gigwa_get_variants() %>% 
#     readGenotypeTableFromGigwa()
genoDataFromGigwa <- geno_provider$gigwa_get_variants(max_missing = 0.2, 
                                                      min_maf = 0.35, 
                                                      samples=geno_samples[1:30]) %>% 
    readGenotypeTableFromGigwa()

In [ ]:
genoDataFromGigwa

## Retrieve phenotype data (BMS)

In [ ]:
pheno_provider$list_crops()

In [ ]:
pheno_provider$set_crop("maize")

In [ ]:
pheno_provider$list_programs()

In [ ]:
pheno_provider$set_program("rTASSEL Testbed")

In [ ]:
pheno_provider$list_trials()

In [ ]:
pheno_provider$set_trial("rTASSEL_Demo_wHapMap")

In [ ]:
pheno_provider$list_studies()

In [ ]:
pheno_provider$set_study(study_name = "rTASSEL_Demo_wHapMap Environment Number 1")

In [ ]:
pheno_provider$get_study_data() %>%
    glimpse()

## Combine genotype and phenotype data

In [ ]:
combinedPenoGeno <- pheno_provider$get_study_data() %>%
    select(germplasmName, EH_M_cm, Q1, Q2, Q3) %>%
    mutate(
        EH_M_cm = as.numeric(EH_M_cm),
        Q1 = as.numeric(Q1),
        Q2 = as.numeric(Q2),
        Q3 = as.numeric(Q3)
    ) %>%
    readGenotypePhenotype(
        genoPathOrObj = genoDataFromGigwa, 
        phenoPathDFOrObj = .,
        taxaID = "germplasmName", 
        attributeTypes = c("data", rep("covariate", 3))
    )

combinedPenoGeno

## Run GWAS

In [ ]:
combinedPenoGeno %>% assocModelFitter(
    EH_M_cm ~ Q1 + Q2 + Q3,
    fitMarkers = TRUE
    ) #%$% GLM_Stats

In [ ]:
# Calculate GLM
tasGLM <- rTASSEL::assocModelFitter(
    tasObj = combinedPenoGeno,             # <- our prior TASSEL object
    formula = EH_M_cm ~ Q1 + Q2 + Q3,
    fitMarkers = TRUE,                 # <- set this to TRUE for GLM
    kinship = NULL,
    fastAssociation = FALSE
)

In [ ]:
str(tasGLM)

In [ ]:
tasGLM@results$GLM_Stats